# Integration via Operating System Processes


This integration mechanism is the most language agnostic. As soon as you have some binary program that you can call from the CLI, you can integrate it with a simialar mechanism in the language of your choice.

![](images/li_os.png)

### A Python Program Calling a Binary via a Separate OS Process

```python
import subprocess as subp
import time


proc = subp.Popen(["./cli_avg", '{"matrix": [[1,2,3], [3,4,5,5]]}'], 
                  stdout=subp.PIPE, stdin=subp.PIPE)

data = None
for line in iter(proc.stdout.readline, b''):
    print('Got result: {}'.format(line))
    data = [float(el) for el in line.replace('[', '').replace(']', '').split(',')]
    break
proc.communicate()

print([2 * el for el in data])
```


### A Nim CLI Program 

```nim-lang
import os
import json
import math
import sequtils
import strutils

type
  Data = object
    matrix: seq[seq[int]]
  
proc main(): seq[float] =
    if paramCount() != 1:
        quit("Usage: " & getAppFilename() & " {\"matrix\": [[1,2,3], [3,4,5,5]]}")
    else:
        let matrixStr = paramStr(1)
        let jsonNode = parseJson(matrixStr)
        let data = json.to(jsonNode, Data)

        var res: seq[float] = @[]
        for vector in data.matrix:
            res.add(sum(vector) / vector.len)

        return res

let valSeq = main()
let strings = valSeq.mapIt(string, $it).join(",")
echo "[" & strings & "]"
```


After compiling the program above with 
```bash
$ nim compile cli_avg.nim
```

Now, you have a binary in the local directory, which is called `cli_avg`. You can call that directly, e.g., via `./cli_avg '{"matrix": [[1,2,3], [3,4,5,5]]}'`.

That is, the same as the above Python program does to interact with the binary:

```bash
$ python call_bin.py
Got result: [2.0,4.25]

[4.0, 8.5]
```


# Your turn!

Take the above `cli_avg` (Linux) binary and call it from the language of your choice. Alternatively, call another CLI program, such as `hostname -I` from the language of your choice and to receive the external IP of your machine.

# Compilation to the Same Level of Abstraction

![](images/li_same_abstraction.png)


## JVM Bytecode for Integration
For exmple interaction in between Java and Groovy, both languages compiling to JVM bytecode.


### A Java Program

```Java
public class Main {
    public static void main(String[] args) {

		int[][] matrix = new int[][]{
		  { 1, 2, 3, 4, 5 },
		  { 9, 8, 7, 6, 5, 4, 3, 1 },
		};
    
        Script script = new Script();
        System.out.println(script.computeAvg(matrix));
    }
}
```

### Calling a Groovy Program

```Groovy
def computeAvg(matrix) {
	println matrix
	def result = []
	matrix.eachWithIndex { a, i ->
  		result.add(a.sum() / a.size())
	}
	return result	
}

def matrix =  [[1, 23, 5, 4], [2, 4, 5, 6, 7, 8]]
println computeAvg(matrix)
```

Since the Groovy program is self-contained, you can run it with: `groovy Script.groovy`.


### Compiling the Sources


After compiling the sources with the following commands

```bash
groovyc Script.groovy
javac -classpath .:$(GROOVY_HOME)/embeddable/groovy-all-2.4.12.jar Main.java
```

you will find three `.class` files

```bash
$ ls -ltrh
-rw-r--r-- 1 vagrant vagrant 2.7K Dec 12 11:08 Script$_computeAvg_closure1.class
-rw-r--r-- 1 vagrant vagrant 3.0K Dec 12 11:08 Script.class
-rw-r--r-- 1 vagrant vagrant  580 Dec 12 11:08 Main.class
```

What is in those `.class` files? Check it for example with `javap -c Main.class`, `javap -c Script.class`, etc.


### Running the Sources

Now, you can execute the compiled program on the JVM with

```bash
java -classpath .:$(GROOVY_HOME)/embeddable/groovy-all-2.4.12.jar Main
```

_Reference:_ This section is based on the following Stackoverflow entry https://stackoverflow.com/a/3990829


## Object Code as Common Level of Integration

Of course you can integrate on any level of abstraction. Above, we integrated our programs on JVM bytecode level. We could do the same on object code level. 

Consider the two following Ada packages, whose functions get called in a C `main` program.

```Ada
package Unit1 is
   function Add (A, B : Integer) return Integer;
   pragma Export (C, Add, "add");
end Unit1;
```

```Ada
package body Unit1 is
   function Add (A, B : Integer) return Integer is
   begin
      return A + B;
   end Add;
end Unit1;
```


```Ada
package Unit2 is
   function Sub (A, B : Integer) return Integer;
   pragma Export (C, Sub, "sub");
end Unit2;
```

```Ada
package body Unit2 is
   function Sub (A, B : Integer) return Integer is
   begin
      return A - B;
   end Sub;
end Unit2;
```
Now, we want to call the two functions from above from C main program.

```C
#include <stdio.h>

extern void adainit (void);
extern void adafinal (void);
extern int add (int, int);
extern int sub (int, int);

int main (int argc, char *argv[])
{
   int a = 21, b = 7;

   adainit();

   /* Should print "21 + 7 = 28" */
   printf ("%d + %d = %d\\n", a, b, add (a, b));

   /* Should print "21 - 7 = 14" */
   printf ("%d - %d = %d\\n", a, b, sub (a, b));

   adafinal();
}
```


Now, you can compile the program by linking the object files together.

```bash
$ gcc -c main.c
$ gnatmake -c unit1.adb
$ gcc-4.9 -c unit1.adb
$ gnatmake -c unit2.adb
$ gcc-4.9 -c unit2.adb
$ gnatbind -n unit1.ali unit2.ali
$ gnatlink unit2.ali main.o -o main
$ ls -ltrh
-rw-r--r-- 1 vagrant vagrant  111 Nov 22 10:27 unit1.ads
-rw-r--r-- 1 vagrant vagrant  125 Nov 22 10:27 unit1.adb
-rw-r--r-- 1 vagrant vagrant  111 Nov 22 10:27 unit2.ads
-rw-r--r-- 1 vagrant vagrant  125 Nov 22 10:28 unit2.adb
-rw-r--r-- 1 vagrant vagrant  170 Nov 22 10:29 Makefile
-rw-r--r-- 1 vagrant vagrant  395 Nov 22 10:37 main.c
-rw-r--r-- 1 vagrant vagrant  387 Dec 12 21:08 unit1.ali
-rw-r--r-- 1 vagrant vagrant 1.9K Dec 12 21:08 main.o
-rw-r--r-- 1 vagrant vagrant 1.3K Dec 12 21:08 unit2.o
-rw-r--r-- 1 vagrant vagrant  387 Dec 12 21:08 unit2.ali
-rw-r--r-- 1 vagrant vagrant 1.3K Dec 12 21:08 unit1.o
-rwxr-xr-x 1 vagrant vagrant  17K Dec 12 21:08 main
```


_Reference:_ This section is based on the GNAT User Manual https://gcc.gnu.org/onlinedocs/gnat_ugn/Interfacing-to-C.html

### How do I see the contents of a binary?

```bash
$ xxd <file>
```

```bash
$ objdump -D -b binary -m i8086 <file>
```

# Your turn!


Try the above type of integration in the langage/technology/framework of choice. Protocoll your steps in a script or a `Makefile`.

  * **JVM** Call for example a bit of Kotlin code from Java
  * **.Net** Try for example to call IronPython from C#, F# from C#

---------------

# Integration via Shared Libraries

![](images/li_so.png)

Let's say we have a bit of C++ code, which we want to make accesible from another language, which knows how to call shared libraries (`.so`/`.dll`) files.

Below, we have a C++ function which can compute the line-wise average of a 2D-numerical array.

```cpp
#include <vector>

std::vector<double> average (std::vector< std::vector<double> > i_matrix);
```

```cpp
#include <vector>
#include "code.h"

using namespace std;

vector<double> average (vector< vector<double> > i_matrix) {

  // Compute average of each row..
  vector <double> averages;
  for (int r = 0; r < i_matrix.size(); r++){
    double rsum = 0.0;
    double ncols= i_matrix[r].size();
    for (int c = 0; c< i_matrix[r].size(); c++){
      rsum += i_matrix[r][c];
    }
    averages.push_back(rsum/ncols);
  }
  return averages;
}
```

### A SWIG Interface

With a tool called SWIG (http://www.swig.org), we can declare which parts of that code should be accesible, e.g., to Ruby and build a shared library out of it.

```swig
%module code
%{
#include "code.h"
%}
%include "std_vector.i"
namespace std {

  /* On a side note, the names VecDouble and VecVecdouble can be changed, but the order of first the inner vector matters! */
  %template(VecDouble) vector<double>;
  %template(VecVecdouble) vector< vector<double> >;
}

%include "code.h"
```

#### SWIG?

  > SWIG is a software development tool that connects programs written in C and C++ with a variety of high-level programming languages. SWIG is used with different types of target languages including common scripting languages such as Javascript, Perl, PHP, Python, Tcl and Ruby. The list of supported languages also includes non-scripting languages such as C#, ..., D, Go language, Java including Android, Lua, Modula-3, OCAML, Octave, Scilab and R.
  > http://www.swig.org


### A Ruby Helper for Building a `Makefile`

```ruby
require 'mkmf'
system('swig -c++ -ruby code.i') or abort
create_makefile('code')
```

### Create a Wrapper and a `Makefile`

```bash
$ ruby extconf.rb  # Create wrapper and Makefile
$ make             # Compile to code.so
$ irb -r./code     # try it
```

### Running the Code

In case you had a file `cli_avg.rb`

```ruby
p Code::average([[1,2,3],[4,5,6,7]])
```

you can run it by adding the library directly from the CLI:

```bash
$ ruby -r./code cli_avg.rb
```




_References:_

This section is based on the following two sources:
  * https://stackoverflow.com/a/23865947
  * http://www.davidegrayson.com/presentations/20140910-swig-ruby/swig-ruby.pdf
  * https://stackoverflow.com/a/9688536
  
-----------------

## Using SWIG Manually

With an interface file, you have to tell SWIG, which parts of your C/C++ code you want to make accessible to another language.

```swig
%module code
%{
#include "code.h"
%}
%include "std_vector.i"
namespace std {

  /* On a side note, the names VecDouble and VecVecdouble can be changed, but the order of first the inner vector matters! */
  %template(VecDouble) vector<double>;
  %template(VecVecdouble) vector< vector<double> >;
}

%include "code.h"
```

Below is the C++ interface and implementation of the code, which we want to call from Python as a normal module.

```cpp
#ifndef _code
#define _code

#include <vector>

std::vector<double> average (std::vector< std::vector<double> > i_matrix);

#endif
```

```cpp
#include <vector>
#include "code.h"

using namespace std;

vector<double> average (vector< vector<double> > i_matrix) {

  // Compute average of each row..
  vector <double> averages;
  for (int r = 0; r < i_matrix.size(); r++){
    double rsum = 0.0;
    double ncols= i_matrix[r].size();
    for (int c = 0; c< i_matrix[r].size(); c++){
      rsum += i_matrix[r][c];
    }
    averages.push_back(rsum/ncols);
  }
  return averages;
}
```


### Building the Code into a Shared Library, which can be imported in Python


```bash
$ ls -ltrh
total 16K
-rw-r--r-- 1 vagrant vagrant 315 Nov 21 11:06 code.i
-rw-r--r-- 1 vagrant vagrant 436 Nov 21 14:11 code.cpp
-rw-r--r-- 1 vagrant vagrant 131 Nov 21 14:11 code.h
-rw-r--r-- 1 vagrant vagrant 266 Nov 21 14:15 Makefile
$ g++ -c -fPIC code.cpp
$ ls -ltr
total 52
-rw-r--r-- 1 vagrant vagrant   315 Nov 21 11:06 code.i
-rw-r--r-- 1 vagrant vagrant   436 Nov 21 14:11 code.cpp
-rw-r--r-- 1 vagrant vagrant   131 Nov 21 14:11 code.h
-rw-r--r-- 1 vagrant vagrant   266 Nov 21 14:15 Makefile
-rw-r--r-- 1 vagrant vagrant 34472 Dec 12 14:30 code.o
$ swig -c++ -python code.i
$ ls -ltr
total 432
-rw-r--r-- 1 vagrant vagrant    315 Nov 21 11:06 code.i
-rw-r--r-- 1 vagrant vagrant    436 Nov 21 14:11 code.cpp
-rw-r--r-- 1 vagrant vagrant    131 Nov 21 14:11 code.h
-rw-r--r-- 1 vagrant vagrant    266 Nov 21 14:15 Makefile
-rw-r--r-- 1 vagrant vagrant  34472 Dec 12 14:30 code.o
-rw-r--r-- 1 vagrant vagrant 373092 Dec 12 14:30 code_wrap.cxx
-rw-r--r-- 1 vagrant vagrant  10575 Dec 12 14:30 code.py
$ g++ -c -fPIC code_wrap.cxx  -I/usr/include/python2.7 -I/usr/lib/python2.7
$ ls -ltr
total 1100
-rw-r--r-- 1 vagrant vagrant    315 Nov 21 11:06 code.i
-rw-r--r-- 1 vagrant vagrant    436 Nov 21 14:11 code.cpp
-rw-r--r-- 1 vagrant vagrant    131 Nov 21 14:11 code.h
-rw-r--r-- 1 vagrant vagrant    266 Nov 21 14:15 Makefile
-rw-r--r-- 1 vagrant vagrant  34472 Dec 12 14:30 code.o
-rw-r--r-- 1 vagrant vagrant 373092 Dec 12 14:30 code_wrap.cxx
-rw-r--r-- 1 vagrant vagrant  10575 Dec 12 14:30 code.py
-rw-r--r-- 1 vagrant vagrant 680088 Dec 12 14:31 code_wrap.o
$ g++ -shared -Wl,-soname,_code.so -o _code.so code.o code_wrap.o
$ ls -ltr
total 1564
-rw-r--r-- 1 vagrant vagrant    315 Nov 21 11:06 code.i
-rw-r--r-- 1 vagrant vagrant    436 Nov 21 14:11 code.cpp
-rw-r--r-- 1 vagrant vagrant    131 Nov 21 14:11 code.h
-rw-r--r-- 1 vagrant vagrant    266 Nov 21 14:15 Makefile
-rw-r--r-- 1 vagrant vagrant  34472 Dec 12 14:30 code.o
-rw-r--r-- 1 vagrant vagrant 373092 Dec 12 14:30 code_wrap.cxx
-rw-r--r-- 1 vagrant vagrant  10575 Dec 12 14:30 code.py
-rw-r--r-- 1 vagrant vagrant 680088 Dec 12 14:31 code_wrap.o
-rwxr-xr-x 1 vagrant vagrant 474808 Dec 12 14:32 _code.so
```

If you now had a Python program, called `cli_avg.py`, as in the following:

```python
import code


a = [[3, 5, 7], [8, 10, 12, 13]]
b = code.average(a)
print(a, b)
```

You could run it via `python cli_avg.py`. Here you have to use Python 2.7 as we were building the shared library for that version (`g++ -c -fPIC code_wrap.cxx  -I/usr/include/python2.7 -I/usr/lib/python2.7`) 

_Reference:_ This section is based on https://stackoverflow.com/a/23865947

  
  
# Your turn!


Use the aboce C++ code together with SWIG to create a language binding for the language of your choice.

Use the official SWIG documentation at http://www.swig.org/Doc3.0/SWIGDocumentation.html.

--------------------------

# Integration by Transpilation

![](images/li_trans.png)



### Modernising an old Fortran Program...


The following Fortran 77 program is taken from here: https://en.wikibooks.org/wiki/Fortran/Fortran_examples#FORTRAN_77

It finds the greatest common divisor read from user input.

```Fortran
*     euclid.f (FORTRAN 77)
*     Find greatest common divisor using the Euclidean algorithm

      PROGRAM EUCLID
        PRINT *, 'A?'
        READ *, NA
        IF (NA.LE.0) THEN
          PRINT *, 'A must be a positive integer.'
          STOP
        END IF
        PRINT *, 'B?'
        READ *, NB
        IF (NB.LE.0) THEN
          PRINT *, 'B must be a positive integer.'
          STOP
        END IF
        PRINT *, 'The GCD of', NA, ' and', NB, ' is', NGCD(NA, NB), '.'
        STOP
      END

      FUNCTION NGCD(NA, NB)
        IA = NA
        IB = NB
    1   IF (IB.NE.0) THEN
          ITEMP = IA
          IA = IB
          IB = MOD(ITEMP, IB)
          GOTO 1
        END IF
        NGCD = IA
        RETURN
      END
```

As you can see in the list of generated files, `fort77` -which relies on `f2c` for the actual conversion into C code-, creates a C program (`GCD.c`). This is in turn compiled and linked into the `gcd` binary.

```bash
$ fort77 GCD.f -o gcd
$ ls -ltrh
-rw-r--r-- 1 vagrant vagrant  166 Nov 22 12:46 main.c
-rw-r--r-- 1 vagrant vagrant 1.6K Nov 22 12:46 main.o
-rw-r--r-- 1 vagrant vagrant 2.9K Dec 12 13:21 GCD.c
-rwxr-xr-x 1 vagrant vagrant 9.5K Dec 12 13:25 gcd
$ ./gcd
 A?
12
 B?
36
 The GCD of 12 and 36 is 12.
```


### Integrating with other Technological Domains

```html
<html>
  <body>
    <script type="text/javascript">
    function addTwoIntegers(a, b)
    {
      return a + b;
    }
    </script>
    <script type="text/javascript" src="calculator.js"></script>
  </body>
</html>
```

```nim
proc addTwoIntegers(a, b: int): int {.importc.}

when isMainModule:
  echo addTwoIntegers(3, 7)
```


```bash
$ nim js -o:calculator.js calculator.nim
```


![](images/nim_js.png)


--------------------------

# Embedding an other Language as String


![](images/li_embed.png)

### SQL Embedded in Java

```java
package dk.cphbusiness.db.neo4j.intro;

import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.SQLException;
import java.sql.Statement;


public class ConnectionTest {

    public static void main(String[] args) {
        Connection con = null;
        Statement st = null;
        ResultSet rs = null;

        String url = "jdbc:mysql://localhost:3306/your_db";
        String user = "root";
        String password = "pwd";

        String query = "SELECT * FROM your_db.t_user";

        try {

            con = DriverManager.getConnection(url, user, password);

            st = con.createStatement();
            rs = st.executeQuery(query);

            if (rs.next()) {                
                System.out.println(rs.getString(1));
            }

        } catch (SQLException ex) {
            System.err.println(ex.getMessage());
            System.err.println(ex);
        } finally { 
            try {
                if (rs != null) {
                    rs.close();
                }
                if (st != null) {
                    st.close();
                }
                if (con != null) {
                    con.close();
                }
            } catch (SQLException ex) {
                System.err.println(ex.getMessage());
                System.err.println(ex);
            }
        }
    }
}
```

### Cypher Embedded in Java


```java
package dk.cphbusiness.db.neo4j.intro;

import org.neo4j.driver.v1.*;

public class ConnectionTest {

    public static void main(String[] args) {
        Driver driver = GraphDatabase.driver( 
                "bolt://localhost:7687", 
                AuthTokens.basic( "neo4j", "class" ) );
        Session session = driver.session();

        // Run a query matching all nodes
        StatementResult result = session.run( 
                "MATCH (s)" +
                "RETURN s.name AS name, s.job AS job");

        while ( result.hasNext() ) {
            Record record = result.next();
            System.out.println( record.get("name").asString() );
        }

        session.close();
        driver.close();
    }
}
```

--------------------------

# Embedding another Language's Interpreter Directly

![](images/li_embed_inter.png)


Some languages are design as extension languages. Lua (http://www.lua.org) is one of such. Usually, you embed the entire Lua interpreter via a library into your program.


In the following a possible Lua script, which is interpreted as a configuration file by the C program below.

```lua
-- configuration file for program `pp'
if getenv("DISPLAY") == ":0.0" then
  width = 300; height = 300
else
  width = 200; height = 200
end
```

```c
#include <lua.h>
#include <lauxlib.h>
#include <lualib.h>

void load (char *filename, int *width, int *height) {
  lua_State *L = lua_open();
  luaopen_base(L);
  luaopen_io(L);
  luaopen_string(L);
  luaopen_math(L);

  if (luaL_loadfile(L, filename) || lua_pcall(L, 0, 0, 0))
    error(L, "cannot run configuration file: %s",
             lua_tostring(L, -1));

  lua_getglobal(L, "width");
  lua_getglobal(L, "height");
  if (!lua_isnumber(L, -2))
    error(L, "`width' should be a number\n");
  if (!lua_isnumber(L, -1))
    error(L, "`height' should be a number\n");
  *width = (int)lua_tonumber(L, -2);
  *height = (int)lua_tonumber(L, -1);

  lua_close(L);
}
```


_Reference:_ This section is based on http://www.lua.org/pil/25.html


# Your turn!

Write a C program with the code from above that consumes the Lua configuration script and prints the product of the two variables `width` and `height`. The product shall be computed in the C host program.

--------------------------